In [3]:
from essentia import *
from essentia.standard import *
import pandas
import os
import numpy as np
import pickle as pkl

list_of_files = os.listdir("./datasets/vaillant_1/audio/")
num_files = list_of_files.__len__()

print('preparing extractor...')
extractor = FreesoundExtractor(profile='custom_extractor_profile.yaml')
features = extractor("./datasets/vaillant_1/audio/" + list_of_files[0])
features = features[0]
output = pandas.DataFrame(index=range(num_files), columns=[features.descriptorNames()], dtype='float')
all_features = features.descriptorNames()

print('extracting features...')
for file in range(num_files):
    if list_of_files[file] == '.DS_Store':
        print('DS_Store item encountered & removed')
        os.remove(list_of_files[file])
    path = "./datasets/vaillant_1/audio/" + list_of_files[file]
    features = extractor(path)
    features = features[0]

    for feat in range(len(all_features)):
        current_feat = str(all_features[feat])
        if isinstance(features[current_feat], float):
            output.iloc[file, feat] = features[str(all_features[feat])]
        else:
            break
    begin_list = feat + 1
    for feat in range(begin_list, len(all_features)):
        current_feature = features[str(all_features[feat])]
        if (type(current_feature) != str) or (not current_feature_mean):
            current_feature_list = current_feature.tolist()
            current_feature_mean = np.mean(current_feature_list)
            if np.isnan(current_feature_mean):
                output.iloc[file, feat] = 0.0
            else:
                output.iloc[file, feat] = current_feature_mean
        else: break
    end_list = feat
    print('extracting file ' + str(file) + ' of ' + str(num_files))

output.dropna(axis=1, inplace = True)

# print('annotate data...')
# #values = np.zeros(23)
# #for i in range(23):
#     #values[i] = random.uniform(0.0, 1.0)
# #values = [0.1,0.2,0.01,0.001,0.24,0.99,0.2,0.12,0.1,0.12,0.21,0.13,0.13,0.12,0.11,0.14,0.15,0.14,0.12,0.16,0.133,0.121,0.141]
# values = pkl.load(open("/Users/simonzimmermann/dev/random_forest_regressor/datasets/pkl/brunnensounds_mean.pkl"))
# target = pandas.Series(data=values, index=range(0,len(values)), dtype=int, name='target')
# output = pandas.concat([output, target], axis=1)

output.to_pickle('test_files_features.pkl')
print(output)

preparing extractor...
extracting features...
extracting file 0 of 34
extracting file 1 of 34
extracting file 2 of 34
extracting file 3 of 34
extracting file 4 of 34
extracting file 5 of 34
extracting file 6 of 34
extracting file 7 of 34
extracting file 8 of 34
extracting file 9 of 34
extracting file 10 of 34
extracting file 11 of 34
extracting file 12 of 34
extracting file 13 of 34
extracting file 14 of 34
extracting file 15 of 34
extracting file 16 of 34
extracting file 17 of 34
extracting file 18 of 34
extracting file 19 of 34
extracting file 20 of 34
extracting file 21 of 34
extracting file 22 of 34
extracting file 23 of 34
extracting file 24 of 34
extracting file 25 of 34
extracting file 26 of 34
extracting file 27 of 34
extracting file 28 of 34
extracting file 29 of 34
extracting file 30 of 34
extracting file 31 of 34
extracting file 32 of 34
extracting file 33 of 34
   lowlevel.average_loudness lowlevel.barkbands_crest.dmean  \
0                   0.039892                       

In [4]:
output


,lowlevel.average_loudness,lowlevel.barkbands_crest.dmean,lowlevel.barkbands_crest.dmean2,lowlevel.barkbands_crest.dvar,lowlevel.barkbands_crest.dvar2,lowlevel.barkbands_crest.max,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.median,lowlevel.barkbands_crest.min,lowlevel.barkbands_crest.stdev,...,tonal.hpcp.var,rhythm.beats_position,rhythm.bpm_histogram,rhythm.bpm_intervals,rhythm.onset_times,tonal.chords_histogram,lowlevel.gfcc.cov,lowlevel.gfcc.icov,lowlevel.mfcc.cov,lowlevel.mfcc.icov
0,0.039892,4.391915,7.086281,10.918573,27.581282,25.752552,19.022726,19.675905,5.208927,4.520816,...,0.043577,0.470204,0.004,0.313469,0.352169,4.166667,28.539422,0.001066,34.269752,-7.593787
1,0.803723,4.564061,7.176280,8.475721,29.404461,23.504385,15.528860,14.370176,4.315845,4.257761,...,0.022768,0.000000,0.000,0.000000,0.011610,4.166667,35.301577,0.001129,58.904350,0.001230
2,0.967106,0.890556,1.245098,1.890924,2.735544,15.828040,14.149800,14.388938,5.488889,1.454563,...,0.004416,0.731429,0.004,0.487619,0.505034,4.166667,12.832060,0.034502,38.061321,0.003748
3,0.952172,2.985121,5.377025,5.243539,13.733944,21.356562,16.429790,17.145454,5.551892,3.032874,...,0.038372,0.522449,0.004,0.348299,0.377324,4.166667,44.097515,0.003865,57.903179,0.003889
4,0.910102,5.059909,7.930267,13.974414,40.793121,23.703365,13.635521,13.187500,5.425776,4.899842,...,0.026594,0.000000,0.000,0.000000,0.470204,4.166667,22.788831,0.002144,30.400288,0.004867
5,0.939820,3.364011,5.400388,6.314789,12.599540,22.406458,16.765596,17.231083,5.171741,3.441147,...,0.047113,0.539864,0.004,0.383129,0.011610,4.166667,23.436076,0.002060,41.040743,0.002052
6,0.960451,1.966941,3.310472,1.474072,4.995806,13.271930,9.473716,9.478817,5.451251,1.642446,...,0.010379,0.731429,0.004,0.487619,0.505034,4.166667,18.322667,0.002874,30.860961,0.004711
7,0.983263,0.783519,0.818484,1.794506,1.124511,15.532306,14.284946,14.566008,4.020636,1.699621,...,0.000718,0.000000,0.000,0.000000,0.505034,4.166667,17.934859,0.034029,56.609300,0.003416
8,0.182911,4.633594,7.553105,10.799868,24.521158,26.594101,20.121044,20.747488,5.290182,4.674016,...,0.030786,0.000000,0.000,0.000000,0.522449,4.166667,32.570044,0.001272,45.413105,0.001274
9,0.987000,0.261396,0.259373,1.103693,0.748564,13.592732,13.250736,13.485076,5.166350,1.243201,...,0.004512,0.731429,0.004,0.487619,0.505034,4.166667,9.433780,0.352472,36.587216,0.007495
